In [76]:
import os

import torch
import wandb
from tqdm.auto import tqdm
import pandas as pd
import pytorch_lightning as pl
import xarray as xr

import eugene as eu
import eugene.train
import eugene.models
import eugene.models.zoo
from eugene import settings
import seqdata as sd

In [4]:
settings.dataset_dir = "/cellar/users/aklie/data/eugene/revision/ray13"
settings.output_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/output/ray13"
settings.logging_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/logs/ray13"
settings.config_dir = "/cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/configs/ray13"
wandb.login()

wandb: Currently logged in as: dlaub. Use `wandb login --relogin` to force relogin


True

In [95]:
sdata = sd.open_zarr('/cellar/users/aklie/data/eugene/revision/ray13/norm_setA_MT.zarr')
target_cols_MT = [n for n in sdata.data_vars.keys() if 'RNCMPT' in n]
sdata = (
    sdata
    .assign(
        target=xr.concat(
            sdata[target_cols_MT].data_vars.values(),
            dim=xr.DataArray(target_cols_MT, dims='_targets')
        )
    )
    .drop(target_cols_MT)
    .transpose('_sequence', '_targets', '_ohe', 'length')
)
sdata

<xarray.Dataset>
Dimensions:    (_sequence: 110645, _ohe: 4, length: 41, _targets: 233)
Coordinates:
  * _targets   (_targets) <U11 'RNCMPT00001' 'RNCMPT00002' ... 'RNCMPT00291'
Dimensions without coordinates: _sequence, _ohe, length
Data variables:
    Probe_ID   (_sequence) object dask.array<chunksize=(27662,), meta=np.ndarray>
    Probe_Set  (_sequence) object dask.array<chunksize=(27662,), meta=np.ndarray>
    RNA_Seq    (_sequence) object dask.array<chunksize=(27662,), meta=np.ndarray>
    ohe_seq    (_sequence, _ohe, length) float16 dask.array<chunksize=(27662, 1, 11), meta=np.ndarray>
    train_val  (_sequence) bool dask.array<chunksize=(110645,), meta=np.ndarray>
    target     (_sequence, _targets) float64 dask.array<chunksize=(27662, 1), meta=np.ndarray>

In [68]:
# Define the architecture to be trained
arch = eu.models.zoo.DeepBind(
    input_len=41,  # Length of padded sequences
    output_dim=len(target_cols_MT),  # Number of multitask outputs
    conv_kwargs=dict(input_channels=4, conv_channels=[1024], conv_kernels=[16], dropout_rates=0.25, batchnorm=0.25),
    dense_kwargs=dict(hidden_dims=[512], dropout_rates=0.25, batchnorm=True),
)

# Initialize the model prior to conv filter initialization
eu.models.init_weights(arch)

# Wrap the model in a SequenceModule
model = eu.models.SequenceModule(
    arch=arch,
    task="regression",
    loss_fxn="mse",
    optimizer="adam",
    optimizer_lr=0.0005,
).cuda()

In [96]:
sdata[['ohe_seq', 'target']].load();

In [97]:
train_sdata = sdata.sel(_sequence=(sdata.train_val == True).compute())
dl = sd.get_torch_dataloader(
    train_sdata,
    sample_dims='_sequence',
    variables=['ohe_seq', 'target'],
    batch_size=2**14,
)

### PyTorch

In [86]:
def train(model, dloader, optim, loss_fn):
    model.train()
    for batch in tqdm(dloader, position=1, leave=False):
        pred = model(batch['ohe_seq'].cuda())
        loss = loss_fn(pred.squeeze(), batch['target'].cuda())
        loss.backward()
        optim.step()
        optim.zero_grad()

In [88]:
epochs = 10
optim = torch.optim.Adam(model.parameters())
with wandb.init(project='EUGENe GPU Utilization', name='Native PyTorch', tags=['Ray13']):
    for _ in tqdm(range(epochs), position=0):
        train(model, dl, optim, model.loss_fxn)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

### Lightning

In [89]:
# Define the architecture to be trained
arch = eu.models.zoo.DeepBind(
    input_len=41,  # Length of padded sequences
    output_dim=len(target_cols_MT),  # Number of multitask outputs
    conv_kwargs=dict(input_channels=4, conv_channels=[1024], conv_kernels=[16], dropout_rates=0.25, batchnorm=0.25),
    dense_kwargs=dict(hidden_dims=[512], dropout_rates=0.25, batchnorm=True),
)

# Initialize the model prior to conv filter initialization
eu.models.init_weights(arch)

# Wrap the model in a SequenceModule
model = eu.models.SequenceModule(
    arch=arch,
    task="regression",
    loss_fxn="mse",
    optimizer="adam",
    optimizer_lr=0.0005,
).cuda()

In [90]:
trainer = pl.Trainer(max_epochs=10, logger=False)

/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.9 /cellar/users/dlaub/miniconda3/envs/eugene-dev/li ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [91]:
with wandb.init(project='EUGENe GPU Utilization', name='PL no val', tags=['Ray13']):
    trainer.fit(model, dl)

/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.9 /cellar/users/dlaub/miniconda3/envs/eugene-dev/li ...
  rank_zero_warn(
/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /cellar/users/dlaub/projects/ML4GLand/EUGENe_paper/notebooks/gpu_utilization_analysis/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and 

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


### EUGENe

In [101]:
# Define the architecture to be trained
arch = eu.models.zoo.DeepBind(
    input_len=41,  # Length of padded sequences
    output_dim=len(target_cols_MT),  # Number of multitask outputs
    conv_kwargs=dict(input_channels=4, conv_channels=[1024], conv_kernels=[16], dropout_rates=0.25, batchnorm=0.25),
    dense_kwargs=dict(hidden_dims=[512], dropout_rates=0.25, batchnorm=True),
)

# Initialize the model prior to conv filter initialization
eu.models.init_weights(arch)

# Wrap the model in a SequenceModule
model = eu.models.SequenceModule(
    arch=arch,
    task="regression",
    loss_fxn="mse",
    optimizer="adam",
    optimizer_lr=0.0005,
).cuda()

In [103]:
# Fit the model
# with wandb.init(project='EUGENe GPU Utilization', name='EUGENe', tags=['Ray13']):
eu.train.fit_sequence_module(
    model,
    sdata,
    seq_key="ohe_seq",
    target_keys=['target'],
    in_memory=True,
    train_key="train_val",
    epochs=10,
    batch_size=2**14,
    drop_last=False,
    early_stopping_metric=None,
    model_checkpoint_monitor=None,
)

Dropping 0 sequences with NaN targets.
Loading ohe_seq and ['target'] into memory
No seed set


/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.9 /cellar/users/dlaub/miniconda3/envs/eugene-dev/li ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type     | Params
------------------------------------------
0 | arch         | DeepBind | 1.2 M 
1 | train_metric | R2Score  | 0     
2 | val_metric   | R2Score  | 0     
3 | test_metric  | R2Score  | 0     
------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.953     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/cellar/users/dlaub/miniconda3/envs/eugene-dev/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of t

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
